### 2. Merge

#### take look at all the cleaned dataset in once 

In [ ]:
import pandas as pd
import copy
collisions_cleaned = pd.read_csv('CleanedData/collisions_cleaned.csv')
intersections_cleaned  = pd.read_csv('CleanedData/intersections_cleaned.csv')
traffic_signals_cleaned  = pd.read_csv('CleanedData/traffic_signals_cleaned.csv')
poles_cleaned  = pd.read_csv('CleanedData/poles_cleaned.csv')
streets_cleaned  = pd.read_csv('CleanedData/streets_cleaned.csv')
signs_cleaned  = pd.read_csv('CleanedData/signs_cleaned.csv')

In [ ]:
collisions = copy.deepcopy(collisions_cleaned)
print("Rows:",len(collisions))
for column in collisions.columns:
    print(f"Column name: {column}, Type: {collisions[column].dtype}")
    #print(f"Unique values: {collisions[column].unique()}")

In [ ]:
intersections = copy.deepcopy(intersections_cleaned)
print("Rows:",len(intersections))
for column in intersections.columns:
    print(f"Column name: {column}, Type: {intersections[column].dtype}")
    #print(f"Unique values: {intersections[column].unique()}")

#Arterial classification code:
#5 - Interstate Freeway
#4 - State Highway
#3 - Collector Arterial
#2 - Minor Arterial
#1 - Principal Arterial
#0 - Not Designated (not an arterial) 

In [ ]:
traffic_signals = copy.deepcopy(traffic_signals_cleaned)
print("Rows:",len(traffic_signals))
for column in traffic_signals.columns:
    print(f"Column name: {column}, Type: {traffic_signals[column].dtype}")
    #print(f"Unique values: {traffic_signals[column].unique()}")

In [ ]:
poles = copy.deepcopy(poles_cleaned)
print("Rows:",len(poles))
for column in poles.columns:
    print(f"Column name: {column}, Type: {poles[column].dtype}")
    #print(f"Unique values: {poles[column].unique()}")

In [ ]:
streets = copy.deepcopy(streets_cleaned)
print("Rows:",len(streets))
for column in streets.columns:
    print(f"Column name: {column}, Type: {streets[column].dtype}")
    #print(f"Unique values: {streets[column].unique()}")

####  Some data visualization

In [ ]:
import folium
# plot all 15469 intersections
streetlights = poles[poles['STREETLIGHT'] == 'YES']

seattle_coordinates = (47.6062, -122.3321)
m = folium.Map(location=seattle_coordinates, zoom_start=12)


for index, row in intersections.iterrows():
    folium.Circle(
        location=[row['Y'], row['X']], 
        radius=15, 
        color='blue', 
        fill=0, 
        fill_color='blue', 
        fill_opacity=0.3, 
    ).add_to(m)

#plot all collision points
for index, row in collisions.iterrows():
    folium.CircleMarker(
        location=[row['Y'], row['X']],
        radius=2,  
        color='red',  
        fill=True,
        fill_color='red', 
        fill_opacity=1, 
    ).add_to(m)

#plot all poles points
for index, row in streetlights.iterrows():
    folium.CircleMarker(
        location=[row['Y'], row['X']],
        radius=2,  
        color='orange',  
        fill=True,
        fill_color='orange', 
        fill_opacity=1, 
    ).add_to(m)

m

In [ ]:
#Some formulas
import pandas as pd
from rtree import index
import math 
from IPython.display import display, clear_output

def haversine(lat1, lon1, lat2, lon2):
    # 将十进制度数转换为弧度
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine公式
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))

    # 地球半径 (单位: 米)
    earth_radius = 6371000
    distance = earth_radius * c
    return distance

def is_point_in_circle(latitude, longitude, center_latitude, center_longitude, radius):
    distance = haversine(latitude, longitude, center_latitude, center_longitude)
    return distance <= radius

def count_collisions(row, idx, collisions, radius):
    global progress_counter
    progress_counter += 1
    clear_output(wait=True)
    display(f"Progress: {progress_counter}/15469")
    
    center_latitude = row['Y']
    center_longitude = row['X']
    nearby_points = list(idx.intersection((center_longitude - radius, center_latitude - radius, center_longitude + radius, center_latitude + radius)))
    count = 0
    
    for point_idx in nearby_points:
        latitude = collisions.at[point_idx, 'Y']
        longitude = collisions.at[point_idx, 'X']
        if is_point_in_circle(latitude, longitude, center_latitude, center_longitude, radius):
            count += 1
    return count


### Merge Collision and Intersection

In [ ]:
radius = 15
progress_counter = 0
idx = index.Index()
for i, row in collisions.iterrows():
    idx.insert(i, (row['X'], row['Y'], row['X'], row['Y']))

intersections_collision['collision_count'] = intersections.apply(lambda row: count_collisions(row, idx, collisions, radius), axis=1)

In [ ]:
intersections_collision.to_csv("MergedData/intersections_collisions.csv", index=False)
intersections_collision.head(10)

### Merge with ploes

In [ ]:
intersections_collision = pd.read_csv('MergedData/intersections_collisions.csv')
streetlights = poles[poles['STREETLIGHT'] == 'YES']

In [ ]:
radius = 15
progress_counter = 0
streetlights_counts = []
streetlights_height_avgs = []

for index, row in intersections_collision.iterrows():
    center_latitude = row['Y']
    center_longitude = row['X']

    streetlights_within_radius = []
    
    progress_counter += 1
    clear_output(wait=True)
    display(f"Progress: {progress_counter}/15469")
    
    for _, streetlight in streetlights.iterrows():
        latitude = streetlight['Y']
        longitude = streetlight['X']

        if is_point_in_circle(latitude, longitude, center_latitude, center_longitude, radius):
            streetlights_within_radius.append(streetlight)

    streetlights_count = len(streetlights_within_radius)
    streetlights_counts.append(streetlights_count)

    streetlights_height_sum = sum([streetlight['HEIGHT'] for streetlight in streetlights_within_radius])
    streetlights_height_avg = streetlights_height_sum / streetlights_count if streetlights_count > 0 else 0
    streetlights_height_avgs.append(streetlights_height_avg)



In [ ]:
intersections_collision_streetlights = intersections_collision
intersections_collision_streetlights['streetlights'] = streetlights_counts
intersections_collision_streetlights['streetlights_height_avg'] = streetlights_height_avgs

intersections_collision_streetlights.to_csv("MergedData/intersections_collision_streetlights.csv", index=False)
intersections_collision_streetlights.head(10)

In [ ]:
intersections_collision_streetlights = pd.read_csv('MergedData/intersections_collision_streetlights.csv')
#check how many different data values of poles_height_avg in intersections_collision_streetlights
unique_values = intersections_collision_streetlights['streetlights_height_avg'].nunique()
print(f'streetlights_height_avg has {unique_values} unique data value。')

### Merge with streets

In [ ]:
streets = copy.deepcopy(streets_cleaned)
print("Rows:",len(streets))
for column in streets.columns:
    print(f"Column name: {column}, Type: {streets[column].dtype}")
    #print(f"Unique values: {streets[column].unique()}")

In [ ]:
streets.head(10)

In [ ]:
#check if the slope are all positive numbers
unique_values = streets['SLOPE_PCT'].unique()
print(unique_values)

In [ ]:
#check if the PVMTCONDINDX1 unique values
unique_values = streets['PVMTCONDINDX1'].unique()
print(unique_values)

In [ ]:
#check How many 0s in PVMTCONDINDX1
value_counts = streets['PVMTCONDINDX1'].value_counts()
zero_count = value_counts.loc[0.0] if 0.0 in value_counts.index else 0
print("Number of 0.0 values:", zero_count)

In [ ]:
#check How many 0s in PVMTCONDINDX2
value_counts = streets['PVMTCONDINDX2'].value_counts()
zero_count = value_counts.loc[0.0] if 0.0 in value_counts.index else 0
print("Number of 0.0 values:", zero_count)

In [ ]:
intersections_collision_streetlights = pd.read_csv('MergedData/intersections_collision_streetlights.csv')

print("Rows:",len(intersections_collision_streetlights))
for column in intersections_collision_streetlights.columns:
    print(f"Column name: {column}, Type: {intersections_collision_streetlights[column].dtype}")
    
intersections_collision_streetlights.head(10)

In [ ]:
unique_values = intersections_collision_streetlights['UNITDESC'].unique()
print(len(unique_values))

In [ ]:
def split_unitdesc(unitdesc):
    streets = unitdesc.split(' AND ')
    return pd.Series([streets[0], streets[1]])

if 'UNITDESC' in intersections_collision_streetlights.columns:
    intersections_collision_streetlights[['Street1', 'Street2']] = intersections_collision_streetlights['UNITDESC'].apply(split_unitdesc)
else:
    print("Error: 'UNITDESC' column not found in the DataFrame")

In [ ]:
intersections_collision_streetlights.head(10)

In [ ]:
import numpy as np

def find_related_streets(row):
    
    if_STNAME_ORD_match = ((streets['STNAME_ORD'] == row['Street1']) | (streets['STNAME_ORD'] == row['Street2']))
    if_XSTRLO_match = (
                       ((streets['XSTRLO'] == row['Street1']) & (streets['STNAME_ORD'] != row['Street1'])) | \
                       ((streets['XSTRLO'] == row['Street2']) & (streets['STNAME_ORD'] != row['Street2']))
                      )
    if_XSTRHI_match = (
                       ((streets['XSTRHI'] == row['Street1']) & (streets['STNAME_ORD'] != row['Street1'])) | \
                       ((streets['XSTRHI'] == row['Street2']) & (streets['STNAME_ORD'] != row['Street2']))
                       )
    
    
    match_count = if_STNAME_ORD_match.astype(int)+(if_XSTRLO_match | if_XSTRHI_match).astype(int)
           
    ge_2_indices = match_count[match_count >= 2].index
    ge_2_indices_list = ge_2_indices.tolist()
    related_streets = streets.loc[ge_2_indices_list]

    return related_streets

def calculate_average_features(row):
    global progress_counter
    
    related_streets = find_related_streets(row)
    
    progress_counter += 1
    clear_output(wait=True)
    display(f"Progress: {progress_counter}/15469")
    
    if not related_streets.empty:
        return pd.Series([related_streets.shape[0],
                          related_streets['PVMTCONDINDX1'].mean(),
                          related_streets['PVMTCONDINDX2'].mean(),
                          related_streets['SLOPE_PCT'].mean(),
                          related_streets['TRANCLASS'].mean(),
                          related_streets['SPEEDLIMIT'].mean()])
    else:
        return pd.Series([0, np.nan, np.nan, np.nan, np.nan, np.nan])

In [ ]:
progress_counter = 0
intersections_collision_streetlights_streets = copy.deepcopy(intersections_collision_streetlights)
intersections_collision_streetlights_streets[['related_street_num','PVMTCONDINDX1_mean', 'PVMTCONDINDX2_mean', 'SLOPE_PCT_mean', 'TRANCLASS_mean', 'SPEEDLIMIT_mean']] = intersections_collision_streetlights_streets.apply(calculate_average_features, axis=1)

In [ ]:
intersections_collision_streetlights_streets.head(10)

In [ ]:
intersections_collision_streetlights_streets["related_street_num"].unique()

In [ ]:
rows_with_6 = intersections_collision_streetlights_streets[intersections_collision_streetlights_streets["related_street_num"] == 6]
rows_with_6

In [ ]:
first_row = rows_with_6.iloc[0]
related_streets = find_related_streets(first_row)
related_streets

In [ ]:
print("Rows:",len(intersections_collision_streetlights_streets))
for column in intersections_collision_streetlights_streets.columns:
    print(f"Column name: {column}, Type: {intersections_collision_streetlights_streets[column].dtype},NAs,{len(intersections_collision_streetlights_streets[column])-len(intersections_collision_streetlights_streets[column].dropna())}")
    

In [ ]:
#Drop NAs
intersections_collision_streetlights_streets.dropna(inplace=True)

print("Rows:",len(intersections_collision_streetlights_streets))
for column in intersections_collision_streetlights_streets.columns:
    print(f"Column name: {column}, Type: {intersections_collision_streetlights_streets[column].dtype},NAs,{len(intersections_collision_streetlights_streets[column])-len(intersections_collision_streetlights_streets[column].dropna())}")
    

In [ ]:
columns_to_drop = ['Street1', 'Street2', 'PVMTCONDINDX2_mean']
intersections_collision_streetlights_streets.drop(columns=columns_to_drop, inplace=True)

In [ ]:
intersections_collision_streetlights_streets.head(3)

In [ ]:
intersections_collision_streetlights_streets.to_csv("MergedData/intersections_collision_streetlights_streets.csv", index=False)

### Merge with Traffic Signals

In [ ]:
intersections_collision_streetlights_streets_traffic_signals = pd.read_csv('MergedData/intersections_collision_streetlights_streets.csv')

In [ ]:
print("Rows:",len(traffic_signals_cleaned))
for column in traffic_signals_cleaned.columns:
    print(f"Column name: {column}, Type: {traffic_signals_cleaned[column].dtype},NAs,{len(traffic_signals_cleaned[column])-len(traffic_signals_cleaned[column].dropna())}")
    

In [ ]:
print("Rows:",len(intersections_collision_streetlights_streets_traffic_signals))
for column in intersections_collision_streetlights_streets_traffic_signals.columns:
    print(f"Column name: {column}, Type: {intersections_collision_streetlights_streets_traffic_signals[column].dtype},NAs,{len(intersections_collision_streetlights_streets_traffic_signals[column])-len(intersections_collision_streetlights_streets_traffic_signals[column].dropna())}")
    

In [ ]:
traffic_signals_cleaned['UNITDESC'] = traffic_signals_cleaned['UNITDESC'].str.rstrip()

In [ ]:
intersections_collision_streetlights_streets_traffic_signals = intersections_collision_streetlights_streets_traffic_signals.merge(traffic_signals_cleaned, on='UNITDESC', how='left')

In [ ]:
#Drop NAs
intersections_collision_streetlights_streets_traffic_signals.dropna(inplace=True)

print("Rows:",len(intersections_collision_streetlights_streets_traffic_signals))
for column in intersections_collision_streetlights_streets_traffic_signals.columns:
    print(f"Column name: {column}, Type: {intersections_collision_streetlights_streets_traffic_signals[column].dtype},NAs,{len(intersections_collision_streetlights_streets_traffic_signals[column])-len(intersections_collision_streetlights_streets_traffic_signals[column].dropna())}")
    

In [ ]:
first_row = intersections_collision_streetlights_streets_traffic_signals.iloc[1]
for column_name, value in zip(intersections_collision_streetlights_streets_traffic_signals.columns, first_row):
    print(f"{column_name}: {value}")


In [ ]:
columns_to_drop = ['ARTERIAL_CLASS'，'INT_SIGNAL_TYPE_CD']
intersections_collision_streetlights_streets_traffic_signals.drop(columns=columns_to_drop, inplace=True)

In [ ]:
intersections_collision_streetlights_streets_traffic_signals.to_csv("MergedData/intersections_collision_streetlights_streets_traffic_signals.csv", index=False)

### Merge with street Sign

In [ ]:
intersections_collision_streetlights_streets_street_sign = pd.read_csv('MergedData/intersections_collision_streetlights_streets.csv')
signs = copy.deepcopy(signs_cleaned)

In [ ]:
signs.head(2)

In [ ]:
intersections_collision_streetlights_streets_street_sign.head(2)

In [ ]:
intersections_collision_streetlights_streets_street_sign["SIGNAL_TYPE"].unique()

In [ ]:
radius = 15
progress_counter = 0
intersections_collision_streetlights_streets_street_sign["stop_sign_quantities"] = 0
intersections_collision_streetlights_streets_street_sign["yield_sign_quantities"] = 0

for index, row in intersections_collision_streetlights_streets_street_sign.iterrows():
    center_latitude = row['Y']
    center_longitude = row['X']

    signs_within_radius = []
    
    progress_counter += 1
    clear_output(wait=True)
    display(f"Progress: {progress_counter}/15469")
    
    for _, sign in signs.iterrows():
        latitude = sign['Y']
        longitude = sign['X']

        if is_point_in_circle(latitude, longitude, center_latitude, center_longitude, radius):
            signs_within_radius.append(sign)
    
    stop_count = sum(1 for sign in signs_within_radius if sign["SIGNTYPE"] == "stop")
    yield_count = sum(1 for sign in signs_within_radius if sign["SIGNTYPE"] == "yield")

    intersections_collision_streetlights_streets_street_sign.loc[index, "stop_sign_quantities"] = stop_count
    intersections_collision_streetlights_streets_street_sign.loc[index, "yield_sign_quantities"] = yield_count

In [ ]:
stop_sign_quantities_unique = intersections_collision_streetlights_streets_street_sign["stop_sign_quantities"].value_counts()
yield_sign_quantities_unique = intersections_collision_streetlights_streets_street_sign["yield_sign_quantities"].value_counts()

print("Unique values and counts for stop_sign_quantities:")
print(stop_sign_quantities_unique)
print("\nUnique values and counts for yield_sign_quantities:")
print(yield_sign_quantities_unique)


In [ ]:
unique_traffic_signals_unitdesc = intersections_collision_streetlights_streets_traffic_signals["UNITDESC"].unique()

filtered_intersections_collision_streetlights_streets_street_sign = intersections_collision_streetlights_streets_street_sign[
    ~intersections_collision_streetlights_streets_street_sign["UNITDESC"].isin(unique_traffic_signals_unitdesc)
]
print(len(filtered_intersections_collision_streetlights_streets_street_sign))

In [ ]:
filtered_intersections_collision_streetlights_streets_street_sign.to_csv("MergedData/intersections_collision_streetlights_streets_street_sign.csv", index=False)